In [ ]:
# !pip install torch
# !pip install torchvision
# !pip install opencv-python

# vwersion : 2.1.2 / 이미 설치되어있었으나 에러로 업글시도
# !pip show numpy

# !pip install --upgrade --force-reinstall numpy
!conda install numpy

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import cv2

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# 데이터 로드
train_data = datasets.ImageFolder(root='./dataset' ,transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

# 간단한 CNN 모델 정의
class RPSModel(nn.Module):
    def __init__(self):
        super(RPSModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(64 * 15 * 15, 128)
        self.fc2 = nn.Linear(128, 3)  # 가위, 바위, 보 클래스

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = RPSModel()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 모델 학습
epochs = 5
for epoch in range(epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")
    
# 실시간 예측 코드
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 전처리 및 예측
    image = cv2.resize(frame, (64, 64))
    image = transforms.ToTensor()(image).unsqueeze(0)
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    
    # 예측 결과 표시
    if predicted == 0:
        text = 'Rock'
    elif predicted == 1:
        text = 'Paper'
    else:
        text = 'Scissors'
        
    cv2.putText(frame, text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Rock Paper Scissors', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

RuntimeError: Numpy is not available